# Play with Trajectory_Dataloader class #2

In [1]:
import os
import pickle
import random
import time
import numpy as np
import torch

In [2]:
DATASET_NAME_TO_NUM = {
    'eth': 0,
    'hotel': 1,
    'zara1': 2,
    'zara2': 3,
    'univ': 4
}

In [3]:
class Object(object):
    pass

args = Object()
args.dataset = 'eth5'
args.test_set = 'zara1'
args.save_dir = './output/' + args.test_set + "/"
args.seq_length = 20
args.obs_length = 20
args.neighbor_thred = 10
args.batch_size = 8
args.batch_around_ped = 256
if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)

self = Object()
self.args = args


In [4]:
if self.args.dataset == 'eth5':

    self.data_dirs = ['../data/eth/univ', '../data/eth/hotel',
                      '../data/ucy/zara/zara01', '../data/ucy/zara/zara02',
                      '../data/ucy/univ/students001', '../data/ucy/univ/students003',
                      '../data/ucy/univ/uni_examples', '../data/ucy/zara/zara03']

    # Data directory where the pre-processed pickle file resides
    self.data_dir = './data'
    skip = [6, 10, 10, 10, 10, 10, 10, 10]

    train_set = [i for i in range(len(self.data_dirs))]

    assert args.test_set in DATASET_NAME_TO_NUM.keys(), 'Unsupported dataset {}'.format(args.test_set)

    args.test_set = DATASET_NAME_TO_NUM[args.test_set]

    if args.test_set == 4 or args.test_set == 5:
        self.test_set = [4, 5]
    else:
        self.test_set = [self.args.test_set]

    for x in self.test_set:
        train_set.remove(x)

    self.train_dir = [self.data_dirs[x] for x in train_set]
    self.test_dir = [self.data_dirs[x] for x in self.test_set]
    self.trainskip = [skip[x] for x in train_set]
    self.testskip = [skip[x] for x in self.test_set]
else:
    raise NotImplementedError
    
self.train_data_file = os.path.join(self.args.save_dir, "train_trajectories.cpkl")
self.test_data_file = os.path.join(self.args.save_dir, "test_trajectories.cpkl")
self.train_batch_cache = os.path.join(self.args.save_dir, "train_batch_cache.cpkl")
self.test_batch_cache = os.path.join(self.args.save_dir, "test_batch_cache.cpkl")

## 1. dataPreprocess

In [5]:
setname = 'train'

In [6]:
if setname == 'train':
    data_dirs = self.train_dir
    data_file = self.train_data_file
else:
    data_dirs = self.test_dir
    data_file = self.test_data_file


def load_dict(data_file):
    f = open(data_file, 'rb')
    raw_data = pickle.load(f)
    f.close()

    frameped_dict = raw_data[0]
    pedtraject_dict = raw_data[1]

    return frameped_dict, pedtraject_dict

In [7]:
frameped_dict, pedtraject_dict = load_dict(data_file)

In [8]:
def dataPreprocess(self, setname):
    '''
    Function to load the pre-processed data into the DataLoader object
    '''
    if setname == 'train':
        val_fraction = 0
        frameped_dict = self.frameped_dict
        pedtraject_dict = self.pedtraject_dict
        cachefile = self.train_batch_cache

    else:
        val_fraction = 0
        frameped_dict = self.test_frameped_dict
        pedtraject_dict = self.test_pedtraject_dict
        cachefile = self.test_batch_cache
        
    if setname != 'train':
        shuffle = False
    else:
        shuffle = True
        
    data_index = self.get_data_index(frameped_dict, setname, ifshuffle=shuffle)
    val_index = data_index[:, :int(data_index.shape[1] * val_fraction)]
    train_index = data_index[:, (int(data_index.shape[1] * val_fraction) + 1):]
    trainbatch = self.get_seq_from_index_balance(frameped_dict, pedtraject_dict, train_index, setname)
    valbatch = self.get_seq_from_index_balance(frameped_dict, pedtraject_dict, val_index, setname)
    trainbatchnums = len(trainbatch)
    valbatchnums = len(valbatch)

    f = open(cachefile, "wb")
    pickle.dump((trainbatch, trainbatchnums, valbatch, valbatchnums), f, protocol=2)
    f.close()

## 2. get_data_index

In [9]:
def get_data_index(self, data_dict, setname, ifshuffle=True):
    '''
    Get the dataset sampling index.
    data_index:
        --> First row: frame id in set
        --> which set
        --> new frame id, all scenes
    '''
    set_id = [] # which scene is the frame of frame_id_in_set related to
    frame_id_in_set = [] # frames in all train/test scenes
    total_frame = 0 # total number of frames in train/test scenes
    
    for seti, dict in enumerate(data_dict):
        frames = sorted(dict)
        maxframe = max(frames) - self.args.seq_length
        ####### TOCHECK
        ####### Why are we subtracting 20 frames and not 20 timesteps?!
        ####!!!!!### maxframe = max(frames) - self.args.seq_length*(frames[1] - frames[0])
        frames = [x for x in frames if not x > maxframe]
        total_frame += len(frames)
        set_id.extend(list(seti for i in range(len(frames))))
        frame_id_in_set.extend(list(frames[i] for i in range(len(frames))))

    all_frame_id_list = list(i for i in range(total_frame))

    data_index = np.concatenate((np.array([frame_id_in_set], dtype=int), np.array([set_id], dtype=int),
                                 np.array([all_frame_id_list], dtype=int)), 0)
    if ifshuffle:
        random.Random().shuffle(all_frame_id_list)
    data_index = data_index[:, all_frame_id_list]

    # to make full use of the data. Add again at the end the fisrt #batch_size frames
    if setname == 'train':
        data_index = np.append(data_index, data_index[:, :self.args.batch_size], 1)
    return data_index

In [10]:
data_index = get_data_index(self, frameped_dict, setname, ifshuffle=False)

In [11]:
print("data_index:")
data_index[:,-10:-1]

data_index:


array([[7500, 7510,  780,  786,  792,  798,  804,  810,  816],
       [   6,    6,    0,    0,    0,    0,    0,    0,    0],
       [6092, 6093,    0,    1,    2,    3,    4,    5,    6]])

## 3. find_trajectory_fragment

In [49]:
def find_trajectory_fragment(trajectory, startframe, seq_length, skip):
    '''
    Query the trajectory fragment based on the index. Replace with 0 if data does not exist.
    '''
    return_trajec = np.zeros((seq_length, 3))
    endframe = startframe + (seq_length) * skip
    start_n = np.where(trajectory[:, 0] == startframe)
    end_n = np.where(trajectory[:, 0] == endframe)
    iffull = False
    ifexsitobs = False

    if start_n[0].shape[0] == 0 and end_n[0].shape[0] != 0:
        start_n = 0
        end_n = end_n[0][0]
        if end_n == 0:
            return return_trajec, iffull, ifexsitobs

    elif end_n[0].shape[0] == 0 and start_n[0].shape[0] != 0:
        start_n = start_n[0][0]
        end_n = trajectory.shape[0]

    elif end_n[0].shape[0] == 0 and start_n[0].shape[0] == 0:
        start_n = 0
        end_n = trajectory.shape[0]

    else:
        end_n = end_n[0][0]
        start_n = start_n[0][0]

    candidate_seq = trajectory[start_n:end_n]
    offset_start = int((candidate_seq[0, 0] - startframe) // skip)

    offset_end = self.args.seq_length + int((candidate_seq[-1, 0] - endframe) // skip)

    return_trajec[offset_start:offset_end + 1, :3] = candidate_seq

    if return_trajec[self.args.obs_length - 1, 1] != 0:
        ifexsitobs = True

    if offset_end - offset_start >= seq_length - 1:
        iffull = True

    return return_trajec, iffull, ifexsitobs

## 4. get_seq_from_index_balance

In [22]:
def get_seq_from_index_balance(self, frameped_dict, pedtraject_dict, data_index, setname):
    '''
    Query the trajectories fragments from data sampling index.
    Notes: Divide the scene if there are too many people; accumulate the scene if there are few people.
    This function takes less gpu memory.
    '''
    batch_data_mass = []
    batch_data = []
    Batch_id = []

    temp = self.args.batch_around_ped
    if setname == 'train':
        skip = self.trainskip
    else:
        skip = self.testskip

    ped_cnt = 0
    last_frame = 0
    for i in range(data_index.shape[1]):
        if i % 100 == 0:
            print("Processed frames:", i, '/', data_index.shape[1])
        cur_frame, cur_set, _ = data_index[:, i]
        framestart_pedi = set(frameped_dict[cur_set][cur_frame])
        try:
            frameend_pedi = set(frameped_dict[cur_set][cur_frame + self.args.seq_length * skip[cur_set]])
        except:
            continue
        present_pedi = framestart_pedi | frameend_pedi
        if (framestart_pedi & frameend_pedi).__len__() == 0:
            continue
        traject = ()
        IFfull = []
        for ped in present_pedi:
            cur_trajec, iffull, ifexistobs = find_trajectory_fragment(pedtraject_dict[cur_set][ped], cur_frame,
                                                                           self.args.seq_length, skip[cur_set])
            if len(cur_trajec) == 0:
                continue
            if ifexistobs == False:
                # Just ignore trajectories if their data don't exsist at the last obversed time step (easy for data shift)
                continue
            if sum(cur_trajec[:, 0] > 0) < 5:
                # filter trajectories have too few frame data
                continue

            cur_trajec = (cur_trajec[:, 1:].reshape(-1, 1, 2),)
            traject = traject.__add__(cur_trajec)
            IFfull.append(iffull)
        if traject.__len__() < 1:
            continue
        if sum(IFfull) < 1:
            continue
        traject_batch = np.concatenate(traject, 1)
        batch_pednum = sum([i.shape[1] for i in batch_data]) + traject_batch.shape[1]

        cur_pednum = traject_batch.shape[1]
        ped_cnt += cur_pednum
        batch_id = (cur_set, cur_frame,)

        if cur_pednum >= self.args.batch_around_ped * 2:
            # too many people in current scene
            # split the scene into two batches
            ind = traject_batch[self.args.obs_length - 1].argsort(0)
            cur_batch_data, cur_Batch_id = [], []
            Seq_batchs = [traject_batch[:, ind[:cur_pednum // 2, 0]], traject_batch[:, ind[cur_pednum // 2:, 0]]]
            for sb in Seq_batchs:
                cur_batch_data.append(sb)
                cur_Batch_id.append(batch_id)
                cur_batch_data = massup_batch(cur_batch_data)
                batch_data_mass.append((cur_batch_data, cur_Batch_id,))
                cur_batch_data = []
                cur_Batch_id = []

            last_frame = i
        elif cur_pednum >= self.args.batch_around_ped:
            # good pedestrian numbers
            cur_batch_data, cur_Batch_id = [], []
            cur_batch_data.append(traject_batch)
            cur_Batch_id.append(batch_id)
            cur_batch_data = massup_batch(cur_batch_data)
            batch_data_mass.append((cur_batch_data, cur_Batch_id,))

            last_frame = i
        else:  # less pedestrian numbers <64
            # accumulate multiple framedata into a batch
            if batch_pednum > self.args.batch_around_ped:
                # enough people in the scene
                batch_data.append(traject_batch)
                Batch_id.append(batch_id)

                batch_data = massup_batch(batch_data)
                batch_data_mass.append((batch_data, Batch_id,))

                last_frame = i
                batch_data = []
                Batch_id = []
            else:
                batch_data.append(traject_batch)
                Batch_id.append(batch_id)

    if last_frame < data_index.shape[1] - 1 and setname == 'test' and batch_pednum > 1:
        batch_data = massup_batch(batch_data)
        batch_data_mass.append((batch_data, Batch_id,))
    self.args.batch_around_ped = temp
    return batch_data_mass

In [14]:
def massup_batch(batch_data):
    '''
    Massed up data fragements in different time window together to a batch
    '''
    num_Peds = 0
    for batch in batch_data:
        num_Peds += batch.shape[1]

    seq_list_b = np.zeros((self.args.seq_length, 0))
    nodes_batch_b = np.zeros((self.args.seq_length, 0, 2))
    nei_list_b = np.zeros((self.args.seq_length, num_Peds, num_Peds))
    nei_num_b = np.zeros((self.args.seq_length, num_Peds))
    num_Ped_h = 0
    batch_pednum = []
    for batch in batch_data:
        num_Ped = batch.shape[1]
        seq_list, nei_list, nei_num = get_social_inputs_numpy(batch)
        nodes_batch_b = np.append(nodes_batch_b, batch, 1)
        seq_list_b = np.append(seq_list_b, seq_list, 1)
        nei_list_b[:, num_Ped_h:num_Ped_h + num_Ped, num_Ped_h:num_Ped_h + num_Ped] = nei_list
        nei_num_b[:, num_Ped_h:num_Ped_h + num_Ped] = nei_num
        batch_pednum.append(num_Ped)
        num_Ped_h += num_Ped
    return (nodes_batch_b, seq_list_b, nei_list_b, nei_num_b, batch_pednum)

In [15]:
def get_social_inputs_numpy(inputnodes):
    '''
    Get the sequence list (denoting where data exsist) and neighboring list (denoting where neighbors exsist).
    '''
    num_Peds = inputnodes.shape[1]

    seq_list = np.zeros((inputnodes.shape[0], num_Peds))
    # denote where data not missing

    for pedi in range(num_Peds):
        seq = inputnodes[:, pedi]
        seq_list[seq[:, 0] != 0, pedi] = 1

    # get relative cords, neighbor id list
    nei_list = np.zeros((inputnodes.shape[0], num_Peds, num_Peds))
    nei_num = np.zeros((inputnodes.shape[0], num_Peds))

    # nei_list[f,i,j] denote if j is i's neighbors in frame f
    for pedi in range(num_Peds):
        nei_list[:, pedi, :] = seq_list
        nei_list[:, pedi, pedi] = 0  # person i is not the neighbor of itself
        nei_num[:, pedi] = np.sum(nei_list[:, pedi, :], 1)
        seqi = inputnodes[:, pedi]
        for pedj in range(num_Peds):
            seqj = inputnodes[:, pedj]
            select = (seq_list[:, pedi] > 0) & (seq_list[:, pedj] > 0)

            relative_cord = seqi[select, :2] - seqj[select, :2]

            # invalid data index
            select_dist = (abs(relative_cord[:, 0]) > self.args.neighbor_thred) | (
                    abs(relative_cord[:, 1]) > self.args.neighbor_thred)

            nei_num[select, pedi] -= select_dist

            select[select == True] = select_dist
            nei_list[select, pedi, pedj] = 0
    return seq_list, nei_list, nei_num

# Execute find_trajectory_fragment

In [161]:
cur_set = 0
ped = 3
cur_frame = 804 # for ped2 --> 804: True-True, 912: False-False, 1024: Error
cur_frame = 912

if setname == 'train':
    skip = self.trainskip
elif setname == 'test':
    skip = self.testskip

In [162]:
cur_trajec, iffull, ifexistobs = find_trajectory_fragment(pedtraject_dict[cur_set][ped], cur_frame,
                                                                           self.args.seq_length, skip[cur_set])
print("iffull:", iffull)
print("ifexistobs:", ifexistobs)
print("trajectory_fragment shape:", cur_trajec.shape)
cur_trajec

iffull: False
ifexistobs: False
trajectory_fragment shape: (20, 3)


array([[ 9.1200e+02,  7.0111e+00,  6.1392e+00],
       [ 9.1800e+02,  7.0778e+00,  5.7418e+00],
       [ 9.2400e+02,  7.1425e+00,  5.3854e+00],
       [ 9.3000e+02,  7.0356e+00,  5.0606e+00],
       [ 9.3600e+02,  6.9620e+00,  4.8729e+00],
       [ 9.4200e+02,  7.0212e+00,  4.6004e+00],
       [ 9.4800e+02,  6.9870e+00,  4.4592e+00],
       [ 9.5400e+02,  7.0495e+00,  4.1372e+00],
       [ 9.6000e+02,  6.9863e+00,  3.7569e+00],
       [ 9.6600e+02,  6.9629e+00,  3.4230e+00],
       [ 9.7200e+02,  6.8523e+00,  3.0802e+00],
       [ 9.7800e+02,  7.0434e+00,  2.8041e+00],
       [ 9.8400e+02,  7.2976e+00,  2.2372e+00],
       [ 9.9000e+02,  7.1490e+00,  1.7815e+00],
       [ 9.9600e+02,  6.9985e+00,  1.3200e+00],
       [ 1.0020e+03,  6.9355e+00,  8.5965e-01],
       [ 1.0080e+03,  6.9255e+00,  2.4332e-01],
       [ 1.0140e+03,  6.8641e+00, -2.8205e-01],
       [ 1.0200e+03,  6.6592e+00, -7.2057e-01],
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00]])

In [158]:
print("Full trajectory:")
pedtraject_dict[cur_set][ped]

Full trajectory:


array([[ 8.3400e+02,  6.6684e+00,  1.2271e+01],
       [ 8.4000e+02,  6.7666e+00,  1.1936e+01],
       [ 8.4600e+02,  6.9144e+00,  1.1448e+01],
       [ 8.5200e+02,  6.7983e+00,  1.0826e+01],
       [ 8.5800e+02,  6.7861e+00,  1.0394e+01],
       [ 8.6400e+02,  6.8606e+00,  9.8409e+00],
       [ 8.7000e+02,  6.8512e+00,  9.3565e+00],
       [ 8.7600e+02,  6.7963e+00,  8.9454e+00],
       [ 8.8200e+02,  6.8699e+00,  8.4109e+00],
       [ 8.8800e+02,  6.8577e+00,  7.9486e+00],
       [ 8.9400e+02,  6.8070e+00,  7.4355e+00],
       [ 9.0000e+02,  6.8356e+00,  6.9627e+00],
       [ 9.0600e+02,  6.9838e+00,  6.5785e+00],
       [ 9.1200e+02,  7.0111e+00,  6.1392e+00],
       [ 9.1800e+02,  7.0778e+00,  5.7418e+00],
       [ 9.2400e+02,  7.1425e+00,  5.3854e+00],
       [ 9.3000e+02,  7.0356e+00,  5.0606e+00],
       [ 9.3600e+02,  6.9620e+00,  4.8729e+00],
       [ 9.4200e+02,  7.0212e+00,  4.6004e+00],
       [ 9.4800e+02,  6.9870e+00,  4.4592e+00],
       [ 9.5400e+02,  7.0495e+00,  4.137

## Execute get_seq_from_index_balance

In [21]:
batch_data_mass = get_seq_from_index_balance(self, frameped_dict, pedtraject_dict, data_index, setname)
trainbatchnums = len(batch_data_mass)
print("Number of bathes:", trainbatchnums)


Frames: 0 / 6102
Frames: 100 / 6102
Frames: 200 / 6102
Frames: 300 / 6102
Frames: 400 / 6102
Frames: 500 / 6102
Frames: 600 / 6102
Frames: 700 / 6102
Frames: 800 / 6102
Frames: 900 / 6102
Frames: 1000 / 6102
Frames: 1100 / 6102
Frames: 1200 / 6102
Frames: 1300 / 6102
Frames: 1400 / 6102
Frames: 1500 / 6102
Frames: 1600 / 6102
Frames: 1700 / 6102
Frames: 1800 / 6102
Frames: 1900 / 6102
Frames: 2000 / 6102
Frames: 2100 / 6102
Frames: 2200 / 6102
Frames: 2300 / 6102
Frames: 2400 / 6102
Frames: 2500 / 6102
Frames: 2600 / 6102
Frames: 2700 / 6102
Frames: 2800 / 6102
Frames: 2900 / 6102
Frames: 3000 / 6102
Frames: 3100 / 6102
Frames: 3200 / 6102
Frames: 3300 / 6102
Frames: 3400 / 6102
Frames: 3500 / 6102
Frames: 3600 / 6102
Frames: 3700 / 6102
Frames: 3800 / 6102
Frames: 3900 / 6102
Frames: 4000 / 6102
Frames: 4100 / 6102
Frames: 4200 / 6102
Frames: 4300 / 6102
Frames: 4400 / 6102
Frames: 4500 / 6102
Frames: 4600 / 6102
Frames: 4700 / 6102
Frames: 4800 / 6102
Frames: 4900 / 6102
Frames: 5000

In [46]:
batch, batch_id = batch_data_mass[0] # first batch
batch_id  # list of tuples of integers, len = 40
batch_abs, seq_list, nei_list, nei_num, batch_pednum = batch  # batch_norm, shift_value are missing

In [53]:

        IFfull.append(iffull)
    if traject.__len__() < 1:
        continue
    if sum(IFfull) < 1:
        continue
    traject_batch = np.concatenate(traject, 1)
    batch_pednum = sum([i.shape[1] for i in batch_data]) + traject_batch.shape[1]

    cur_pednum = traject_batch.shape[1]
    ped_cnt += cur_pednum
    batch_id = (cur_set, cur_frame,)

    if cur_pednum >= self.args.batch_around_ped * 2:
        # too many people in current scene
        # split the scene into two batches
        ind = traject_batch[self.args.obs_length - 1].argsort(0)
        cur_batch_data, cur_Batch_id = [], []
        Seq_batchs = [traject_batch[:, ind[:cur_pednum // 2, 0]], traject_batch[:, ind[cur_pednum // 2:, 0]]]
        for sb in Seq_batchs:
            cur_batch_data.append(sb)
            cur_Batch_id.append(batch_id)
            cur_batch_data = massup_batch(cur_batch_data)
            batch_data_mass.append((cur_batch_data, cur_Batch_id,))
            cur_batch_data = []
            cur_Batch_id = []

        last_frame = i
    elif cur_pednum >= self.args.batch_around_ped:
        # good pedestrian numbers
        cur_batch_data, cur_Batch_id = [], []
        cur_batch_data.append(traject_batch)
        cur_Batch_id.append(batch_id)
        cur_batch_data = massup_batch(cur_batch_data)
        batch_data_mass.append((cur_batch_data, cur_Batch_id,))

        last_frame = i
    else:  # less pedestrian numbers <64
        # accumulate multiple framedata into a batch
        if batch_pednum > self.args.batch_around_ped:
            # enough people in the scene
            batch_data.append(traject_batch)
            Batch_id.append(batch_id)

            batch_data = massup_batch(batch_data)
            batch_data_mass.append((batch_data, Batch_id,))

            last_frame = i
            batch_data = []
            Batch_id = []
        else:
            batch_data.append(traject_batch)
            Batch_id.append(batch_id)

if last_frame < data_index.shape[1] - 1 and setname == 'test' and batch_pednum > 1:
    batch_data = massup_batch(batch_data)
    batch_data_mass.append((batch_data, Batch_id,))
self.args.batch_around_ped = temp
# return batch_data_mass

Processed frames: 0 / 6102
Processed frames: 100 / 6102
Processed frames: 200 / 6102
Processed frames: 300 / 6102
Processed frames: 400 / 6102
Processed frames: 500 / 6102
Processed frames: 600 / 6102
Processed frames: 700 / 6102
Processed frames: 800 / 6102
Processed frames: 900 / 6102
Processed frames: 1000 / 6102
Processed frames: 1100 / 6102
Processed frames: 1200 / 6102
Processed frames: 1300 / 6102
Processed frames: 1400 / 6102
Processed frames: 1500 / 6102
Processed frames: 1600 / 6102
Processed frames: 1700 / 6102
Processed frames: 1800 / 6102
Processed frames: 1900 / 6102
Processed frames: 2000 / 6102
Processed frames: 2100 / 6102
Processed frames: 2200 / 6102
Processed frames: 2300 / 6102
Processed frames: 2400 / 6102
Processed frames: 2500 / 6102
Processed frames: 2600 / 6102
Processed frames: 2700 / 6102
Processed frames: 2800 / 6102
Processed frames: 2900 / 6102
Processed frames: 3000 / 6102
Processed frames: 3100 / 6102
Processed frames: 3200 / 6102
Processed frames: 3300

KeyboardInterrupt: 

In [57]:
# def get_seq_from_index_balance(
# self, frameped_dict, pedtraject_dict, data_index, setname):

batch_data_mass = []
batch_data = []
Batch_id = []

temp = self.args.batch_around_ped
if setname == 'train':
    skip = self.trainskip
else:
    skip = self.testskip

ped_cnt = 0
last_frame = 0

# loop over frames
for i in range(2):
    if i % 100 == 0:
        print("Processed frames:", i, '/', data_index.shape[1])
    cur_frame, cur_set, _ = data_index[:, i]
    # pedestrians in the start frame
    framestart_pedi = set(frameped_dict[cur_set][cur_frame])
    try:
        # pedestrians in the end frame (+20 frames)
        frameend_pedi = set(frameped_dict[cur_set][cur_frame + self.args.seq_length * skip[cur_set]])
    except:
        # skip if no pedestrians or any problem
        continue
    # pedestrians present in the currents 20 frames
    present_pedi = framestart_pedi | frameend_pedi
    
    ######
    
    # if the initial pedestrians all disappers after 20 frames
    if (framestart_pedi & frameend_pedi).__len__() == 0:
        continue
    traject = ()
    IFfull = []
    # loop over the pedestrians in the start frame (and possibly beyond)
    for ped in present_pedi:
        cur_trajec, iffull, ifexistobs = find_trajectory_fragment(pedtraject_dict[cur_set][ped], cur_frame,
                                                                       self.args.seq_length, skip[cur_set])
        if len(cur_trajec) == 0:
            continue
        if ifexistobs == False:
            # Just ignore trajectories if their data don't exsist at the last obversed time step (easy for data shift)
            continue
        if sum(cur_trajec[:, 0] > 0) < 5:
            # filter trajectories have too few frame data
            continue

        cur_trajec = (cur_trajec[:, 1:].reshape(-1, 1, 2),)
        traject = traject.__add__(cur_trajec)
        IFfull.append(iffull)
    
    

Processed frames: 0 / 6102


In [71]:
IFfull

[True, True, True, True, True, True, True, False, False]

In [73]:
traject[0]

array([[[5.1734, 6.0058]],

       [[5.1737, 6.0054]],

       [[5.1737, 6.0048]],

       [[5.1739, 6.0044]],

       [[5.1739, 6.0037]],

       [[5.1739, 6.0033]],

       [[5.1741, 6.0027]],

       [[5.1741, 6.0023]],

       [[5.1744, 6.0016]],

       [[5.1744, 6.001 ]],

       [[5.1746, 6.0006]],

       [[5.1746, 5.9999]],

       [[5.1746, 5.9995]],

       [[5.1749, 5.9989]],

       [[5.1749, 5.9985]],

       [[5.1751, 5.9978]],

       [[5.1751, 5.9972]],

       [[5.1753, 5.9968]],

       [[5.1753, 5.9962]],

       [[5.1753, 5.9957]]])